In [7]:
import pandas as pd
import random
import numpy as np
 
#количество пациентов
num_patients = 50

patient_ids = [f'P{i+1}' for i in range(num_patients)]

#Различные медицинские параметры 
temperatures = [round(random.uniform(36.5, 39.5), 1) for _ in range(num_patients)]
heart_rates = [random.randint(60, 100) for _ in range(num_patients)]
white_blood_cell_counts = [random.randint(4000, 11000) for _ in range(num_patients)]
flu_symptoms = [random.choice(['Yes', 'No']) for _ in range(num_patients)]

data = {
    'Patient_ID': patient_ids,
    'Temperature (Celsius)': temperatures,
    'Heart Rate (bpm)': heart_rates,
    'White Blood Cell Count': white_blood_cell_counts,
    'Flu Symptoms': flu_symptoms
}

df = pd.DataFrame(data)

print(df.head())

  Patient_ID  Temperature (Celsius)  Heart Rate (bpm)  White Blood Cell Count  \
0         P1                   39.1                99                   10289   
1         P2                   37.3                86                    9665   
2         P3                   36.6                61                    5299   
3         P4                   38.5                84                    9285   
4         P5                   38.2                76                    8351   

  Flu Symptoms  
0          Yes  
1           No  
2          Yes  
3           No  
4          Yes  


In [8]:
from sklearn.model_selection import train_test_split

#задаем целевую переменную
target_variable = 'Flu Symptoms'

#разделяем данные на признаки(X) и целевую переменную(y)
X = df.drop(target_variable, axis=1)
y = df[target_variable]

#разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Размер обучающего набора:", X_train.shape)
print("Размер тестового набора:", X_test.shape)


Размер обучающего набора: (40, 4)
Размер тестового набора: (10, 4)


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train_no_id = X_train.drop('Patient_ID', axis=1)
X_test_no_id = X_test.drop('Patient_ID', axis=1)

#создаеи модель RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

#обучаем модель на обучающем наборе
rf_model.fit(X_train_no_id, y_train)

#предсказываем значения для тестового набора
y_pred_rf = rf_model.predict(X_test_no_id)

#оцениваем производительност модели RandomForestClassifier
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
classification_rep_rf = classification_report(y_test, y_pred_rf)

# Выводим результаты
print(f"Точность модели RandomForestClassifier: {accuracy_rf:.2f}")
print("\nМатрица ошибок:")
print(conf_matrix_rf)
print("\nОтчет о классификации:")
print(classification_rep_rf)


Точность модели RandomForestClassifier: 0.70

Матрица ошибок:
[[6 1]
 [2 1]]

Отчет о классификации:
              precision    recall  f1-score   support

          No       0.75      0.86      0.80         7
         Yes       0.50      0.33      0.40         3

    accuracy                           0.70        10
   macro avg       0.62      0.60      0.60        10
weighted avg       0.68      0.70      0.68        10


In [18]:
#улучшаем произволительность данных, т.к 70 это мало для медицинских прогнозов
from sklearn.model_selection import GridSearchCV

# сетка параметров
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestClassifier(random_state=42)

#Используем Grid Search для настройки гиперпараметров
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_no_id, y_train)

#Получаем лучшие параметры
best_params = grid_search.best_params_

#Обучаем модель с лучшими параметрами
best_rf_model = RandomForestClassifier(random_state=42, **best_params)
best_rf_model.fit(X_train_no_id, y_train)

#Предсказываем значения для тестового набора
y_pred_best_rf = best_rf_model.predict(X_test_no_id)

#Оцениваем производительность модели с лучшими параметрами
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
conf_matrix_best_rf = confusion_matrix(y_test, y_pred_best_rf)
classification_rep_best_rf = classification_report(y_test, y_pred_best_rf)

#результаты
print(f"Лучшие параметры: {best_params}")
print(f"Точность модели RandomForestClassifier (с лучшими параметрами): {accuracy_best_rf:.2f}")
print("\nМатрица ошибок:")
print(conf_matrix_best_rf)
print("\nОтчет о классификации:")
print(classification_rep_best_rf)


Лучшие параметры: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Точность модели RandomForestClassifier (с лучшими параметрами): 0.80

Матрица ошибок:
[[7 0]
 [2 1]]

Отчет о классификации:
              precision    recall  f1-score   support

          No       0.78      1.00      0.88         7
         Yes       1.00      0.33      0.50         3

    accuracy                           0.80        10
   macro avg       0.89      0.67      0.69        10
weighted avg       0.84      0.80      0.76        10
